In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  2 15:56:55 2017

@author: Runjing Liu
"""
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from copy import deepcopy

from beta_process_vb_lib import *

%matplotlib inline

# np.random.seed(50)



In [2]:
N = 500 # sample size
D = 2 # dimension
# so X will be a n\times D matrix

K_inf = 5 # truncation parameter for sampling

alpha = 5 # IBP parameter
v = np.zeros(K_inf) # Beta sticks
Pi = np.zeros(K_inf)
Z = np.zeros([N,K_inf])

# Parameters to draw A from MVN
mu = np.zeros(D)
sigma_A = 1

sigma_eps = 0.5 # variance of noise

# Draw Z from truncated stick breaking process
for k in range(K_inf):
    v[k] = np.random.beta(alpha,1)
    if k != 0:
        Pi[k] = Pi[k-1] * v[k]
    else:
        Pi[k] = v[k]

    for n in range(N):
        Z[n,k] = np.random.binomial(1,Pi[k])

print(Z[0:10,:])

# Draw A from multivariate normal
A = np.random.multivariate_normal(mu, sigma_A*np.identity(D), K_inf)

# draw noise
epsilon = np.random.multivariate_normal(np.zeros(D), sigma_eps*np.identity(D), N)

# the observed data
X = np.dot(Z,A) + epsilon

K = 5 # variational truncation



[[ 1.  0.  1.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  0.]
 [ 0.  1.  0.  1.  0.]
 [ 0.  1.  0.  1.  0.]
 [ 1.  0.  1.  1.  0.]
 [ 1.  0.  1.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  1.]
 [ 1.  1.  0.  0.  0.]]


In [ ]:
# Variational parameters
tau = np.random.uniform(0,1,[K,2]) # tau1, tau2 -- beta parameters for v
nu = np.random.uniform(0,1,[N,K]) # Bernoulli parameter for z_nk
#nu = deepcopy(Z)
phi_mu = np.random.normal(0,1,[D,K]) # kth mean (D dim vector) in kth column
#phi_mu = deepcopy(A).T
phi_var = {k: np.identity(D) for k in range(K)}


    # Term7 and term 5 give NaNs. Term7 bc nu is close to 1; term 5 bc tau's are large!?
iterations = 100
elbo = np.zeros(iterations)
Term1 = np.zeros(iterations)
Term2 = np.zeros(iterations)
Term3 = np.zeros(iterations)
Term4 = np.zeros(iterations)
Term5 = np.zeros(iterations)
Term6 = np.zeros(iterations)
Term7 = np.zeros(iterations)

In [ ]:
#print(A)
#input('pause')

for i in np.arange(iterations):


    for k in np.arange(K):
        tau = Tau_updates(tau, nu, alpha, D, N, K, n, k)

    for k in np.arange(K):
        Expectation_k = Exp_true(tau, k)
        for n in np.arange(N):
            nu = Nu_updates(Expectation_k, tau, nu, phi_mu, phi_var, sigma_eps, X, D, N, K, n, k)

    for k in np.arange(K):
        [phi_var, phi_mu] = Phi_updates(nu, phi_mu, phi_var, X, sigma_A, sigma_eps, D, N, K, n, k)

    #print('true')
    #print(A)

    #print('estimate')
    #print(phi_mu.T)
    #input('pause')

    [elbo[i], Term1[i], Term2[i], Term3[i], Term4[i], Term5[i], Term6[i], Term7[i]] = \
        Elbo(tau, nu, phi_mu, phi_var, X, sigma_A, sigma_eps, alpha, D, K, N)
    print('iteration: ', i)
    print('ELbo')
    print(elbo[i])

    #print('Z')
    #print(Z[0:10,:])

    #print('nu')
    round_nu = np.round(nu * (nu >= 0.9) + nu * (nu <= 0.1)) + nu * (nu >= 0.1) * (nu <= 0.9)

    print(round_nu[0:10, :])

    # input("paused")

    if np.abs(elbo[i] - elbo[i - 1]) <= 10^(-5):
        break



('iteration: ', 0)
ELbo
-14317.8613781
[[ 0.24743521  0.          0.89710145  0.          0.23460363]
 [ 0.30422068  0.18683119  0.83531798  0.79855736  0.36728439]
 [ 1.          1.          0.          0.          0.        ]
 [ 1.          0.39693534  0.          0.          0.        ]
 [ 1.          0.67930467  0.          0.          0.        ]
 [ 1.          0.66523807  0.          0.          0.25127766]
 [ 0.39069404  0.30175457  0.62809363  0.22696214  0.82069782]
 [ 0.22484483  0.18827158  0.6470385   0.          0.70274749]
 [ 0.          0.27371071  0.89233645  0.47477133  0.57104523]
 [ 1.          0.          0.88874944  0.          0.28602984]]
('iteration: ', 1)
ELbo
-9934.80719969
[[ 0.11043749  0.20893515  0.8939826   0.          0.2283431 ]
 [ 0.          0.43842622  0.51929913  0.77661094  0.62125297]
 [ 1.          1.          0.          0.26842494  0.        ]
 [ 1.          0.74788099  0.          0.          0.        ]
 [ 1.          0.60187267  0.          

In [ ]:

plt.figure(1)
plt.clf()
plt.plot(np.arange(iterations), elbo, linewidth=2.0)
plt.plot(np.arange(iterations), Term1)
plt.plot(np.arange(iterations), Term2)
plt.plot(np.arange(iterations), Term3)
plt.plot(np.arange(iterations), Term4)
plt.plot(np.arange(iterations), Term5)
plt.plot(np.arange(iterations), Term6)
plt.plot(np.arange(iterations), Term7)
plt.xlabel('iteration')
plt.ylabel('elbo')
plt.legend(['elbo', 'Term1', 'Term2', 'Term3', 'Term4', 'Term5', 'Term6', 'Term7'])
